In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np 


result = list(Path("../llm-eval-gpt4/speech").rglob("*.csv"))
result = [str(x) for x in result]
result

In [ ]:
def get_result(files): 
    metrics = ['flip_label', 'minimal_change', 'realistic']#, 'grammar', 'cohesive', 'likability']

    modes = ['mean']#, 'max', 'min']
    dict = {
        'LLM' : [],
        'avg' : [],
    }

    for m in metrics:
        for mode in modes:
            dict[m + ' - ' + mode] = []


    for f in files:
        llm_name = f.split('/')[-2]

        if 'wrong' in llm_name:
            continue 
        df = pd.read_csv(f)
        dict['LLM'].append(llm_name)
        for m in metrics:
            for mode in modes:
                if mode == 'mean':
                    dict[m + ' - ' + mode].append(df[m].mean().round(2))
                if mode == 'max':
                    dict[m + ' - ' + mode].append(df[m].max().round(2))
                if mode == 'min':
                    dict[m + ' - ' + mode].append(df[m].min().round(2))
        for mode in ['mean']:
            dict['avg'].append(np.mean([dict[m + ' - ' + mode][-1] for m in metrics if  'minimal_change_2' not in m]))

    df_results = pd.DataFrame.from_dict(dict)
    df_results['avg'] = df_results['avg'].round(2)
    return df_results.sort_values(by=['LLM'])

In [ ]:
mean_results = get_result(result)

In [ ]:
#mean_results.to_csv('./tables/eval_sent.csv', index=False)

In [ ]:
table = mean_results[['LLM', 'flip_label - mean', 'minimal_change - mean', 'realistic - mean', 'avg']]
print(table.to_latex(index=False))

In [ ]:
table

In [ ]:
(3.66+2.95+2.58)/3

In [ ]:
def get_distribution(files, attribute = 'flip_label'): 
    metrics = [1.0, 2.0, 3.0, 4.0]

    dict = {
        'LLM' : []
    }

    for m in metrics:
        dict[str(m)] = []


    for f in files:
        llm_name = f.split('/')[-2]
        dict['LLM'].append(llm_name)

        df = pd.read_csv(f)
        counts = df[attribute].value_counts(normalize=True)

        for m in metrics:
            try:
                dict[str(m)].append(counts[m]*100)
            except KeyError:
                dict[str(m)].append(0)
    

    df_results = pd.DataFrame.from_dict(dict)
    for m in metrics:
        df_results[str(m)] = df_results[str(m)].round(2)

    return df_results.sort_values(by=['LLM'])

In [ ]:
get_distribution(result, attribute='flip_label')

In [ ]:
get_distribution(result, attribute='minimal_change')

In [ ]:
get_distribution(result, attribute='realistic')